In [59]:
import pandas as pd
import numpy as np
#Reading the parquet file using pandas
df = pd.read_parquet('C:/Users/Arpita/Downloads/yellow_tripdata_2022-01.parquet',engine="auto",columns=None)
#Taking the subset of first 70 rowws of the given data
#df=trips.head(70)
#print(df)

In [73]:
df['frequency'] = float(1)

rateidtypes = {1:"RatecodeID-1", 2:"RatecodeID-2", 3:"RatecodeID-3", 4:"RatecodeID-4", 5:"RatecodeID-5",6: "RatecodeID-6",99:"RatecodeID-99"}

df['RCID']= df['RatecodeID'].map(rateidtypes)

In [84]:
from math import pi

from bokeh.io import show
from bokeh.models import (AnnularWedge, ColumnDataSource,
                          Legend, LegendItem, Plot, Range1d)


xdr = Range1d(start=-2, end=2)
ydr = Range1d(start=-2, end=2)

plot = Plot(x_range=xdr, y_range=ydr)
plot.title.text = "Total amount paid in relation with RatecodeID (Qualitative)"
plot.toolbar_location = None

colors = {
    "RatecodeID-1": "seagreen",
    "RatecodeID-2": "tomato",
    "RatecodeID-3": "orchid",
    "RatecodeID-4": "firebrick",
    "RatecodeID-5": "skyblue",
    "RatecodeID-6": "lightgray",
    "RatecodeID-99": "pink"
}

selected = df.groupby("RCID").sum("numeric_only=True")
print(selected)

               VendorID  passenger_count  trip_distance  RatecodeID  \
RCID                                                                  
RatecodeID-1    3898192        3182253.0     5999670.67   2296363.0   
RatecodeID-2     115868         102033.0     1161604.94    133246.0   
RatecodeID-3       6925           6591.0       65536.76     12042.0   
RatecodeID-4       5458           4553.0       65614.52     12472.0   
RatecodeID-5      24808          19980.0       64696.82     67805.0   
RatecodeID-6         29             29.0          10.26       102.0   
RatecodeID-99      8746           8728.0       58670.55    864468.0   

               PULocationID  DOLocationID  payment_type  fare_amount  \
RCID                                                                   
RatecodeID-1      383395688     377047250       2824911  25764070.37   
RatecodeID-2        9386151      10217236         82619   3373973.16   
RatecodeID-3         634235        178849          5635    247994.00   


In [90]:
rcid = selected.index.tolist()

angles = selected.tolls_amount.map(lambda x: 2*pi*(x/1000000)).cumsum().tolist()

rcid_source = ColumnDataSource(dict(
    start  = [0] + angles[:-1],
    end    = angles,
    colors = [colors[i] for i in rcid],
))

glyph = AnnularWedge(x=0, y=0, inner_radius=0.9, outer_radius=1.8,
                     start_angle="start", end_angle="end",
                     line_color="white", line_width=3, fill_color="colors")
r= plot.add_glyph(rcid_source, glyph)

legend = Legend(location="center")
for i, name in enumerate(colors):
    legend.items.append(LegendItem(label=name, renderers=[r], index=i))
plot.add_layout(legend, "center")

show(plot)